In [263]:
import twstock
import twstock.stock as stock
#import matplotlib.pyplot as plt
import pandas as pd
import json
import sqlite3 
from datetime import datetime

# 讀取所有股票號碼

In [264]:
def get_stockids():
    sids = []
    twse = twstock.twse
    for sid in twse.keys():
        if twse[sid].type == '股票':
            sids.append(sid)
    return sids

# 讀取股票資料

In [265]:
def read_stock(sid:str,strdate=None):
    if strdate == None:
        strdate = datetime.now().strftime("%Y%m%d")

    conn = sqlite3.connect('%s/%s.db' % (strdate,sid),detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()

    # Read table
    sqlite_data = cursor.execute('SELECT * FROM stocks').fetchall()
    
    data_pd = pd.DataFrame(sqlite_data,columns=['date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction'])
    
    return data_pd

# 計算所需股票資訊

In [266]:
def get_info(sid:str,strdate=None):
    stock_pd = read_stock(sid,strdate)
    stock_pd = stock_pd.set_index('date')

    stock_ma3 = stock_pd.rolling(3).mean()
    stock_ma5 = stock_pd.rolling(5).mean()
    stock_ma8 = stock_pd.rolling(8).mean()
    stock_ma20 = stock_pd.rolling(20).mean()
    
    data = pd.concat([stock_ma3['close'],stock_ma5['close'],stock_ma8['close']],axis=1)    
    std = data.std(axis=1)
    mean = data.mean(axis=1)
    
    close = stock_pd[['close']]
    capacity = stock_pd[['capacity']] /1000          
    
    data = {}
    data['norstd'] = std / mean
    data['close'] = close
    data['capacity'] = capacity
    data['ma20'] = stock_ma20['close']
    data['slop'] = 0.5*(stock_pd['close'].iloc[-1] - stock_pd['close'].iloc[-3])
    return data

# 選股策略

In [267]:
def select(strdate=None):
    threshold = 0.002
    ma20_factor = 0.01        # 10% make sure the price just over/below ma20
    capacity_bound = 2000
    bigcapa_factor = 1.5

    sids = get_stockids()
        
    buy=''
    sell=''
    for sid in sids:
        try:
            # 讀取資料
            data = get_info(sid,strdate)
        
            # 必要篩選條件
            bound = data['capacity'].iloc[-1] > capacity_bound
            if data['norstd'][-1] < threshold and bound['capacity']:

                # 爆大量分析
                capacond = data['capacity'].iloc[-1] > bigcapa_factor * data['capacity'].iloc[-2]
                if capacond['capacity']:
                    capamark='爆大量'
                else:
                    capamark=''

                # 股票基本資訊
                stockinfo = '%4.4s %6.5f %6.2f %7.2f %6d %-8.8s %3s'% \
                        (sid,data['norstd'][-1],data['close'].iloc[-1],data['ma20'].iloc[-1], \
                          data['capacity'].iloc[-1],twse[sid].name,capamark)

                # 股票篩選機制
                ma20diff = data['close'].iloc[-1] - data['ma20'].iloc[-1]
                if  ma20diff['close'] > 0 and ma20diff['close'] < ma20_factor*data['ma20'][-1] and data['slop'] > 0:
                    buy  =  buy + ' BUY: ' + stockinfo + '\n'
                elif  ma20diff['close'] < 0 and ma20diff['close'] > -ma20_factor*data['ma20'][-1] and data['slop'] < 0:
                    sell = sell + 'SELL: ' + stockinfo + '\n'
        except:
            pass
            #print('%-8s %-4s Calculate failed'%(twse[sid].name,sid))

    print('%3s %3s%6s%6s%6s%5s%3s'%('買賣','股號','離散度','股價','20日均價','成交量','股名'))
    print(buy)
    print(sell)

In [268]:
select('20180801')

 買賣  股號   離散度    股價 20日均價  成交量 股名




In [269]:
select()

 買賣  股號   離散度    股價 20日均價  成交量 股名
 BUY: 1504 0.00138  24.75   24.60   2402 東元          
 BUY: 2002 0.00031  23.15   23.09   9648 中鋼          
 BUY: 2610 0.00143   9.23    9.18   5013 華航          
 BUY: 2801 0.00119  20.30   20.13   6969 彰銀          


